In [ ]:
from pathlib import Path
from scipy.signal import find_peaks
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from mpl_toolkits.axes_grid1 import make_axes_locatable
import h5py
import copy
import pickle

# Project modules
from lib.Stokes import Stokes
from functions.plot_data import plot_data
from functions.plot_angle_gradient import plot_angle_gradient

In [ ]:
# Open data files
from functions.load_pickles import load_pickles
stokes_list, _ = load_pickles(select="stokes")

In [ ]:
# Extract each Stokes parameter into dictionary, to make it easier to work with
I = stokes_list['I']
Q = stokes_list['Q']
U = stokes_list['U']
V = stokes_list['V']

In [ ]:
# Define function to sum data along the third axis (sum all wavelengths)
def sw(data):
    return np.sum(data, axis=2)

In [ ]:
# Calculate the vertical (longitudinal) component

#NOTE: these values are divided into _1 for the first spectral line and _2 for the second spectral line
lambda0  = [6301.51, 6302.50]  # in Angstroms
gbar     = [1.669, 2.487] # Lozitsky
C1 = [4.6686e-13 * l**2 * g for l, g in zip(lambda0, gbar)]
print(f"C1_1: {C1[0]}, C1_2: {C1[1]}")

f = 1 # filling factor, assumed to be 1

line_cuttoff = 55 # index at which we divide the data, between the spectral lines



# Calculate the horizontal (transverse) component

Gbar = [g**2 for g in gbar]  # Landi Degl'Innocenti & Landolfi (2004), si la línea es un triplete #TODO
C2 = [5.4490e-26 * l**4 * g for l, g in zip(lambda0, Gbar)]
print(f"C2_1: {C2[0]}, C2_2: {C2[1]}")



# Strong field approximation
C = 4.67e-13 # TODO: where is this constant from? Sara's email (and PDF) 29/08


In [ ]:
Bv = [-sw((V.data_n * I.data_d)[:, :, :line_cuttoff]) / (C1[0] * f * sw((I.data_d ** 2)[:, :, :line_cuttoff])),
       -sw((V.data_n * I.data_d)[:, :, line_cuttoff:]) / (C1[1] * f * sw((I.data_d ** 2)[:, :, line_cuttoff:]))]

print(f'Mean Bv [G]:\nFirst line: {np.mean(Bv[0])}\nSecond line: {np.mean(Bv[1])}')

In [ ]:
L = np.sqrt(Q.data_n**2 + U.data_n**2)
Bt = [sw(L[:,:,:line_cuttoff] * np.abs(I.data_dd[:,:,:line_cuttoff])) / (C2[0] * f * sw(np.abs(I.data_dd[:,:,:line_cuttoff]))**2),
       sw(L[:,:,line_cuttoff:] * np.abs(I.data_dd[:,:,line_cuttoff:])) / (C2[1] * f * sw(np.abs(I.data_dd[:,:,line_cuttoff:]))**2)]
Bt = np.sqrt(Bt)


print(f'Mean Bt [G]:\nFirst line: {np.mean(Bt[0])}\nSecond line: {np.mean(Bt[1])}')

In [ ]:
def noise_level(a):
    a = np.asanyarray(a)
    sd = a.std(ddof=1)
    return sd

# Example spectrum noise level
spectrum = copy.copy(V.data_n[300,300,:])
sd = noise_level(spectrum[90:])
print(f'Noise level = {sd}')

In [ ]:
# Calculate delta lambda_B for each pixel, for each line

dlB = np.zeros(I.data.shape)[:,:,:2].astype(float)
dlB_binary = np.zeros(I.data.shape)[:,:,:2].astype(float)
margin = 3.5
verbose = 0

for i in range(0, np.shape(V.data_n)[0]):

    # Read out every 50 rows
    if np.mod(i,50) == 0:
        print(f'Row {i} of {np.shape(V.data_n)[0]}')

    for j in range(0, np.shape(V.data_n)[1]):
        if (verbose):
            print(f'For pixel {i},{j}:')

        # Initialize variables
        peaks_p = [0, 0]
        peaks_n = [0, 0]

        # Get the spectrum for this pixel
        spectrum = copy.copy(V.data_n[i,j,:])

        # Calculate noise level for spectrum region outside the spectral lines
        sd = noise_level(spectrum[90:])
        if (verbose):
            print(f'Noise level = {sd}')

        # First line
        peaks_p[0] = int(np.argmax(spectrum[:60])) if spectrum[:60].max() > margin*sd else None
        peaks_n[0] = int(np.argmin(spectrum[:60])) if spectrum[:60].min() < -margin*sd else None

        # # Second line
        peaks_p[1] = 60 + int(np.argmax(spectrum[60:])) if spectrum[60:].max() > margin*sd else None
        peaks_n[1] = 60 + int(np.argmin(spectrum[60:])) if spectrum[60:].min() < -margin*sd else None

        if (verbose):
            print(f'Peak positions: {peaks_p}, {peaks_n}')

        # If two peaks have been found for the first line, calculate dlB
        if peaks_p[0] is not None and peaks_n[0] is not None:
            dlB_binary[i, j, 0] = 1
            dlB[i, j, 0] = np.array(V.wave_array)[peaks_p[0]] - np.array(V.wave_array)[peaks_n[0]]

            if (verbose):
                plt.figure()
                plt.vlines(V.wave_array[peaks_n[0]], ymin=np.min(spectrum), ymax=np.max(spectrum), color='g', linestyle='--')
                plt.vlines(V.wave_array[peaks_p[0]], ymin=np.min(spectrum), ymax=np.max(spectrum), color='purple', linestyle='--')

                print(f'Peak 1 positive: {V.wave_array[peaks_p[0]]}')
                print(f'Peak 1 negative: {V.wave_array[peaks_n[0]]}')
                print(np.array(V.wave_array)[peaks_p[0]] - np.array(V.wave_array)[peaks_n[0]])
                print(f'Distance: {dlB[i, j, 0]:.6f} Angstrom')
        else:
            dlB[i, j, 0] = np.nan


        # If two peaks have been found for the second line, calculate dlB
        if peaks_p[1] is not None and peaks_n[1] is not None:
            dlB_binary[i, j, 1] = 1
            dlB[i, j, 1] = np.array(V.wave_array)[peaks_p[1]] - np.array(V.wave_array)[peaks_n[1]]

            if (verbose):
                plt.figure()
                plt.vlines(V.wave_array[peaks_n[1]], ymin=np.min(spectrum), ymax=np.max(spectrum), color='g', linestyle='--')
                plt.vlines(V.wave_array[peaks_p[1]], ymin=np.min(spectrum), ymax=np.max(spectrum), color='purple', linestyle='--')

                print(f'Peak 2 positive: {V.wave_array[peaks_p[1]]}')
                print(f'Peak 2 negative: {V.wave_array[peaks_n[1]]}')
                print(np.array(V.wave_array)[peaks_p[1]] - np.array(V.wave_array)[peaks_n[1]])
                print(f'Distance: {dlB[i, j, 1]:.6f} Angstrom')
        else:
            dlB[i, j, 1] = np.nan

        if (verbose):
            plt.hlines(margin*sd, V.wave_array[0], V.wave_array[-1], color='red', linestyle='--')
            plt.hlines(-margin*sd, V.wave_array[0], V.wave_array[-1], color='red', linestyle='--')
            plt.plot(V.wave_array, spectrum, linestyle='-', color='black')

In [ ]:
plot_data(dlB[:,:,0], title=r"$\Delta \lambda _B$, in $\mathrm{\AA}$")

In [ ]:
plot_data(Bv[:,:,0], title=r"$B$, in G$")

In [ ]:
# Calculate strong magnetic field
B_strong = dlB / (2 * C * np.array(lambda0)**2 * np.array(gbar))

In [ ]:
Bv = np.moveaxis(Bv, 0, -1)
Bt = np.moveaxis(Bt, 0, -1)

In [ ]:
# Constants
kB = 1.3806488e-16 # [erg K-1]
h = 6.6260755e-27  # [erg s]
c = 2.99792458e10  # [cm · s−1]
lambda0  = np.array([6301.51*1e-8, 6302.50*1e-8])   # Angstroms to cm
l = np.mean(lambda0)
Icont = I.data_n[:,:,:5].mean(axis=2) # I map in continuum
Teff = 5780 # [K] T quiet sun average
T = (1/Teff - kB*l/(h*c) * np.log(Icont))**-1
M = 55.845 # Fe atomic mass, [g mol-1]
av = 6.022e23 # avogadro, [mol-1]
m =  M/av
Xi = 0 # microturbulence, assumed 0

# Rebecca Centeno, equation 3
dlD = np.array([6301.5/c * np.sqrt(2*kB*T/m + Xi**2), 6302.5/c * np.sqrt(2*kB*T/m + Xi**2)]) # will be in units of lambda, in this case Angstrom

dlD = np.moveaxis(dlD, 0, -1)
np.shape(dlD)
# plot_data(dlD[:,:,0], title=r"$\Delta \lambda _D$, in $\mathrm{\AA}$")

In [ ]:
# remove outliers
B_strong[B_strong > 5000] = np.nan
B_strong[B_strong < -5000] = np.nan

In [ ]:
B = np.empty(Bv.shape)
B[:,:,:] = np.nan
B_mask = np.zeros(Bv.shape) * np.nan
# gamma = np.zeros(derived.weak.Bv.shape) * np.nan
# chi = np.zeros(derived.weak.Bv.shape) * np.nan

for i in range(I.data.shape[0]):
    for j in range(I.data.shape[1]):
        for k in range(0,2):
            if (dlB_binary[i,j,k] == 0) or np.isnan(dlB[i,j,k]) or (np.abs(dlB[i,j,k]) < np.abs(dlD[i,j,k])):
                # If dlB could not be calculated, or if it is less than the doppler effect
                B[i,j,k] = np.sqrt(Bv[i,j,k]**2 + Bt[i,j,k]**2)
                B_mask[i,j,k] = 0
                # chi[i,j,k] = derived.weak.chi[i,j,k]
                # gamma[i,j,k] = derived.weak.gamma[i,j,k]
            else:
                # Use strong field approximation otherwise
                B[i,j,k] = B_strong[i,j,k]
                B_mask[i,j,k] = 1
                # chi[i,j,k] = derived.strong.chi[i,j,k]
                # gamma[i,j,k] = derived.strong.gamma[i,j,k]

In [ ]:
print(np.abs(dlD[300,300,0]))
print(np.abs(dlB[300,300,0]))

In [ ]:
# Histogram of derived.strong.B[:,:,0]
plt.figure()
plt.hist(dlB[:,:,0].ravel(), bins=100, color='blue', alpha=0.7)
plt.xlabel('Magnetic field strength (first line)')
plt.ylabel('Count')
plt.title('Histogram of dlB[:,:,0]')
plt.show()

In [ ]:
# Histogram of derived.strong.B[:,:,0]
plt.figure()
plt.hist(B[:,:,0].ravel(), bins=100, color='blue', alpha=0.7)
plt.xlabel('Magnetic field strength (first line)')
plt.ylabel('Count')
plt.title('Histogram of B[:,:,0]')
plt.show()

In [ ]:
# remove outliers
dlB[dlB > 0.27] = np.nan
dlB[dlB < -0.27] = np.nan

In [ ]:
plot_data(dlB[:,:,0], colourmap='berlin', title=r"$\Delta \lambda _B$, 6301.5 $\mathrm{\AA}$, in $\mathrm{\AA}$")

In [ ]:
plot_data(B[:,:,0], colourmap='PuOr', title=r"B combined [G]")

In [ ]:
plot_data(dlB[:,:,0] > dlD[:,:,0], colourmap='grey')